# Step 1 - Prepare network data
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

This notebook downloads bicycle and street network data sets and prepares them for analysis.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-09  
Last modified: 2020-08-31

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

To check for nominatimstring:
* Go to e.g. https://nominatim.openstreetmap.org/search.php?q=paris%2C+france&polygon_geojson=1&viewbox= and enter the search string. If a correct polygon (or multipolygon) pops up it should be fine.

To get shapefiles:
* Go to [Overpass](overpass-turbo.eu), to the city, and run:
    `relation["boundary"="administrative"]["name:en"="Copenhagen Municipality"]({{bbox}});(._;>;);out skel;`
* Export: Download as GPX
* Use QGIS to create a polygon, with Vector > Join Multiple Lines, and Processing Toolbox > Polygonize (see https://gis.stackexchange.com/questions/98320/connecting-two-line-ends-in-qgis-without-resorting-to-other-software and https://gis.stackexchange.com/questions/207463/convert-a-line-to-polygon)

### Functions

In [ ]:
%run -i functions.py

## Download and wrangle data

### Networks

In [ ]:
for placeid, placeinfo in cities.items():
    if placeinfo["nominatimstring"] != '':
        location = ox.gdf_from_place(placeinfo["nominatimstring"])
        location = shapely.geometry.shape(location['geometry'][0])
    else:
        # https://gis.stackexchange.com/questions/113799/how-to-read-a-shapefile-in-python
        shp = fiona.open(PATH["data"] + placeid + "/" + placeid + ".shp")
        first = shp.next()
        location = shapely.geometry.shape(first['geometry'])
    
    Gs = {}
    for parameterid, parameterinfo in osmnxparameters.items():
        try:
            Gs[parameterid] = ox.graph_from_polygon(location, 
                                   network_type = parameterinfo['network_type'],
                                   custom_filter = (parameterinfo['custom_filter']),
                                   retain_all = True,
                                   simplify = False)
        except:
            Gs[parameterid] = nx.empty_graph(create_using = nx.MultiDiGraph)
            print(placeid + ": No OSM data for graph " + parameterid + ". Created empty graph.")
        if parameterinfo['export']: ox_to_csv(Gs[parameterid], PATH["data"] + placeid + "/", placeid, parameterid)

    # Compose special cases biketrack, bikeable, biketrackcarall
    parameterid = 'biketrack'
    Gs[parameterid] = nx.compose_all([Gs['bike_cyclewaylefttrack'], Gs['bike_cyclewaytrack'], Gs['bike_highwaycycleway'], Gs['bike_bicycledesignated'], Gs['bike_cyclewayrighttrack']])
    ox_to_csv(Gs[parameterid], PATH["data"] + placeid + "/", placeid, parameterid)
    
    parameterid = 'bikeable'
    Gs[parameterid] = nx.compose(Gs['biketrack'], Gs['car30']) 
    ox_to_csv(Gs[parameterid], PATH["data"] + placeid + "/", placeid, parameterid)
    
    parameterid = 'biketrackcarall'
    Gs[parameterid] = nx.compose(Gs['biketrack'], Gs['carall']) # Order is important
    ox_to_csv(Gs[parameterid], PATH["data"] + placeid + "/", placeid, parameterid)
    
    if debug:
        print(len(Gs['carall'].vs), len(Gs['carall'].es))
        print(len(Gs['biketrack'].vs), len(Gs['biketrack'].es))
        print(len(Gs['biketrackcarall'].vs), len(Gs['biketrackcarall'].es))
        print(len(Gs['biketrackcarall'].vs)-len(Gs['biketrack'].vs), len(Gs['biketrackcarall'].es)-len(Gs['biketrack'].es))
        
    for parameterid in networktypes:
        ox_to_csv(ox.simplify_graph(Gs[parameterid]), PATH["data"] + placeid + "/", placeid, parameterid, "_simplified")

In [ ]:
Audio(sound_file, autoplay=True)